In [4]:
from models import *
from utility import *
import logging
import os
import yaml
from pathlib import Path
import plotly
from adapter import OstiMongoAdapter
import pandas as pd
from typing import Union, List, Dict
import plotly.express as px
import plotly.graph_objects as go

In [5]:
# configuration stuff
config_file = Path(os.path.abspath('')).parent / "files" / "config.yaml"
config = yaml.load(open(config_file.as_posix(), 'r'), Loader=yaml.SafeLoader)

# prepare
oma = OstiMongoAdapter.from_config(config)
elink = ConnectionModel.parse_obj(config["osti"]["elink"])
explorer = ConnectionModel.parse_obj(config["osti"]["explorer"])
osti = OSTIModel(elink=elink, explorer=explorer)
oma.materials_store.connect()
oma.doi_store.connect()

In [6]:
def mongo_to_panda(mongo_store, 
                   criteria:dict = {}, 
                   properties:Union[Dict, List, None]=None,
                   skip:int=0, limit:int=0, no_id=True):
    cursor = list(mongo_store.query(criteria=criteria, properties=properties, skip=skip, limit=limit))
    df = pd.DataFrame(list(cursor))
    if no_id:
        del df['_id']
    return df

In [7]:
def make_doi_status():
    df = mongo_to_panda(oma.doi_store)
    labels=df.status.unique()
    values = df.status.value_counts()
    fig=go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5, name="status")])
    fig.update_layout(annotations=[dict(text="Status", font_size=20, showarrow=False)])
    return fig
def make_doi_valid():
    df = mongo_to_panda(oma.doi_store)
    labels=df.valid.unique()
    values = df.valid.value_counts()
    fig=go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5, name="valid")])
    fig.update_layout(annotations=[dict(text="Valid", font_size=20, showarrow=False)])
    return fig
def make_doi_last_updated():
    df = mongo_to_panda(oma.doi_store)
    df.last_updated = [f"{t.year}/{t.month}/{t.day}" for t in df.last_updated] # only get the days
    labels=df.last_updated.unique()
    values = df.last_updated.value_counts()
    fig=go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5, name="last_updated")])
    fig.update_layout(annotations=[dict(text="Last Updated", font_size=20, showarrow=False)])
    return fig
def make_doi_bibtex():
    df = mongo_to_panda(oma.doi_store)
    bibtex_bool = ["Does not have Bibtex" if i == None else "Has Bibtex" for i in df.bibtex]
    df["bibtex_bool"] = bibtex_bool
    labels=df.bibtex_bool.unique()
    values = df.bibtex_bool.value_counts()
    fig=go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5, name="bibtex")])
    fig.update_layout(annotations=[dict(text="bibtex", font_size=20, showarrow=False)])
    return fig

In [8]:
def make_doi_citation_created():
    labels = ["DOI Citation Not Created", "DOI Citation Pending", "DOI Citation Completed"]
    pending_len = len([i for i in oma.doi_store.query(criteria={"status":"PENDING"})])
    values = [oma.materials_store.count()-oma.doi_store.count(),
              pending_len,
              oma.doi_store.count() - pending_len
             ]
    fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.5)])
    fig.update_layout(annotations=[dict(text="DOI Citation", font_size=20, showarrow=False)])
    return fig

In [12]:
fig = make_doi_citation_created()
fig.show()

In [10]:
cursor = list(oma.doi_store.query())
df = pd.DataFrame(list(cursor))
df

,_id,material_id,doi,bibtex,status,valid,last_updated
0,5ec98ba3f3951ad25c7063d7,mp-1173151,10.17188/1479772,None,PENDING,True,2020-05-23 13:46:22.356
1,5ec9eb7bf3951ad25c706609,mp-1194956,10.17188/1479783,None,PENDING,True,2020-05-23 20:35:17.398
2,5ec9ebacf3951ad25c706619,mp-1204405,10.17188/1479784,None,PENDING,True,2020-05-23 20:36:05.280
3,5ec9ebc4f3951ad25c706625,mp-556728,10.17188/1479785,None,PENDING,True,2020-05-23 20:36:28.676
4,5ec9eddcf3951ad25c7066cd,mp-542374,10.17188/1479786,None,PENDING,True,2020-05-23 20:46:15.203
5,5ec9eddcf3951ad25c7066ce,mp-1247854,10.17188/1479787,None,PENDING,True,2020-05-23 20:46:15.203
6,5ec9eddcf3951ad25c7066cf,mp-1212517,10.17188/1479788,None,PENDING,True,2020-05-23 20:46:15.203
7,5ec9eddcf3951ad25c7066d0,mp-10231,10.5072/1322654,None,COMPLETED,True,2020-05-23 20:46:15.203
8,5ec9eddcf3951ad25c7066d1,mp-759337,10.17188/1479789,None,PENDING,True,2020-05-23 20:46:15.203
9,5ec9eddcf3951ad25c7066d2,mp-15472,10.5072/1322184,None,COMPLETED,True,2020-05-23 20:46:15.203
